UNBXD Interns Assignment : Ankit Kumar

In [15]:
import pandas as pd
import json
import requests
import numpy as np
import math

APIs

API for getting the Product results

https://search.unbxd.io/fb853e3332f2645fac9d71dc63e09ec1/demounbxd700181503576558/search?&q=*&rows=<count of products>&start=<0,1,2…>

E.g.https://search.unbxd.io/fb853e3332f2645fac9d71dc63e09ec1/demounbxd700181503576558/search?&q=*&rows=10&start=0 to get first set of results
https://search.unbxd.io/fb853e3332f2645fac9d71dc63e09ec1/demounbxd700181503576558/search?&q=*&rows=10&start=1 to get next set of results


In [16]:
def extract(n):
  base_url = 'https://search.unbxd.io/fb853e3332f2645fac9d71dc63e09ec1/demo-unbxd700181503576558/search?&q=*&rows=10&start=' + n

  return requests.get(base_url).json()

In [17]:
p = extract('1')
list(p)

['searchMetaData', 'response', 'facets']

In [18]:
dta = []
for i in p['response']['products']:
  dta.append(i)

In [19]:
df = pd.DataFrame(dta)
df.head(10)

,startDate,onlineExclusive,colorName,unbxd_color_mapping,color,domainUrl,variantUpc,categoryType,productUrl,selling_price,inStoreInventoryCount,category,availability,promoMessage,size,sku,productImage,backOrderable,productDescription,catlevel2Name,displaySalePrice,title,promote_metric,description,colorCode,onlineExclusivePromoMsg,categoryAll,valid,limitedQuantity,unbxd_color_for_category,sortPrice,type,newProduct,productId,catlevel3,onlineInventoryCount,is_oos_product,category_meta_data,price,neckline,...,recs_availability,gender,imageUrl,productInventory,colors,categoryId,parentProductId,parentProduct,salePrice,giftable,petites,sleeveLength,displayPrice,test_colors,listPrice,uniqueId,timeStamp_unbxd,unbxdFeedId,categoryPath1,categoryPath1_fq,categoryPath,3_unbxd_bool,3_unbxd_sku,autosuggest,autosuggest_unstemmed,3_unbxd_text,1_unbxd_text,doctype,2_unbxd_text,_version_,parent_unbxd,occasion,fit,pattern,trend,catlevel4,catlevel4Name,wash,rise,legShape
0,20151112.0,true,"[Harvest Yellow, Ivory, burnt amber, Pitch Bla...",[Orange::https://images.express.com/is/image/e...,"[Purple, Orange, White, Black, Yellow]",//www.express.com/clothing/women/sleeveless-tu...,"[0096_09605830_0383, 0096_09605830_0383, 0096_...",Sweaters,/clothing/women/sleeveless-turtleneck-sweater/...,[34.93],213.0,"[ALL WOMEN'S REG PRODUCT, Sweaters & Cardigans...",true,Price Reflects 30% Off,"[XXS, XL, XXS Petite, M, L Petite, L, XS Petit...",9605830,https://images.express.com/is/image/expressfas...,"[False, False, False, False, False, False, Fal...",This form-fitting ribbed sleeveless turtleneck...,Sweaters & Cardigans,$34.93,sleeveless turtleneck sweater,true,This form-fitting ribbed sleeveless turtleneck...,"[1545, 3, 383, 2422, 58]",Online Exclusive!,"[ALL WOMEN'S REG PRODUCT, Sweaters & Cardigans...",true,false,"[Thornberry::0096_09605830_0383::false, Pitch ...",34.93,Sweaters,false,9605830,[Sweaters & Cardigans],9902.0,false,"{""cat2012"": {""totalProductCount"": 95, ""categor...",49.9,Turtleneck & Mock Neck,...,true,[women],[https://images.express.com/is/image/expressfa...,9902,"[{""color"": ""Harvest Yellow"", ""skuUpc"": ""0096_9...","[cat4320003, cat2012, cat3970001, cat3960003, ...",9605830,true,$34.93,false,false,Sleeveless,$49.90,"[Harvest Yellow::0096_9605830_2422::False, Pit...",$49.90,9605830,1584713861308,9dbd1262-0541-45c7-8d56-2efe44e5f3ac,"[cat3970001, cat3960003, cat1730035, cat2012, ...","[cat3970001, cat3960003, cat1730035, cat2012, ...","[cat3970001, cat3960003, cat1730035, cat2012, ...",[true],"[19437484, 19437781, 18309553, 82369992, 82370...",sleeveless turtleneck sweater,sleeveless turtleneck sweater,"[0096_09605830_0383, Sweaters & Cardigans, 009...","[burnt amber, Thornberry, Black, L, Orange, M,...",POPULAR_PRODUCTS,"[cat3970001, Sweaters & Cardigans, cat3960003,...",1661692969336438784,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20181212.0,true,"[Ivory, Radiant Peony, Pitch Black, Navy Blue,...",[Yellow::https://images.express.com/is/image/e...,"[Blue, Pink, Yellow, Black, White, Red]",//www.express.com/clothing/women/slim-ruffle-c...,"[0097_09704543_0813, 0097_09704543_0003, 0097_...",Tops,/clothing/women/slim-ruffle-covered-button-por...,[34.93],13168.0,"[Tops, Shirts & Blouses, Portofino Shirts, Spe...",true,Price Reflects 30% Off,"[XXS, XL, XXS Petite, M, L Petite, L, XS Petit...",09704543,https://images.express.com/is/image/expressfas...,"[False, False, False, False, False, False, Fal...","Smooth fabric, feminine ruffles and covered bu...",Tops,$34.93,slim ruffle covered button portofino shirt,true,"Smooth fabric, feminine ruffles and covered bu...","[58, 3, 2177, 179, 2167, 813, 2500]",Online Exclusive!,"[Tops, Shirts & Blouses, Portofino Shirts, Spe...",true,false,"[Navy Blue::0097_09704543_0813::false, Ivory::...",34.93,Shirts,false,09704543,"[Shirts & Blouses, Portofino Shirts, Petite Tops]",8874.0,false,"{""cat4320003"": {""totalProductCount"": 231, ""cat...",49.9,Button Down,...,true,[women],[https://images.express.com

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 95 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   startDate                 10 non-null     float64
 1   onlineExclusive           10 non-null     object 
 2   colorName                 10 non-null     object 
 3   unbxd_color_mapping       10 non-null     object 
 4   color                     10 non-null     object 
 5   domainUrl                 10 non-null     object 
 6   variantUpc                10 non-null     object 
 7   categoryType              10 non-null     object 
 8   productUrl                10 non-null     object 
 9   selling_price             10 non-null     object 
 10  inStoreInventoryCount     10 non-null     float64
 11  category                  10 non-null     object 
 12  availability              10 non-null     object 
 13  promoMessage              10 non-null     object 
 14  size         

In [21]:
df3 = df.copy()

In [22]:
temp = df3['parent_unbxd'][0]
print(temp == True) # x == x is false only for nan

True


In [23]:
def removeDuplicates(arr):
  res = []
  [res.append(x) for x in arr if x not in res]
  converted_list = [str(element) for element in res]
  return converted_list

In [25]:
def process(page):
  dta = []
  jso = extract(str(page))
  for param in jso['response']['products']:
    dta.append(param)
  df2 = pd.DataFrame(dta)
  for column in df2:
    for i in range(len(df2[column])):
      if df2[column][i] == df2[column][i]: # check if the value is not nan
        obj = df2[column][i]

        if type(obj)==str:
          if obj.lower() == 'true':
            df2[column][i] = 'YES'
          if obj.lower() == 'false':
            df2[column][i] = 'NO'
        
        if column == 'parent_unbxd':   # problem in this part
          if str(obj).lower() == 'true':
            df2[column][i] = 'YES'
          if str(obj).lower() == 'false':
            df2[column][i] = 'NO'


        if type(obj) == list:
          unique = removeDuplicates(obj)

          if column == 'unbxd_color_for_category':
            new_unique = [ele.split('::')[0] for ele in unique]
            unique = new_unique
          
          join_str = ','.join(unique)

          ## Processing for true or false
          if join_str.lower() == 'false':
            join_str = 'NO'
          elif join_str.lower() == 'true':
            join_str = 'YES'
          
          df2[column][i] = join_str
  return df2


In [26]:
df0 = process('0')
df0.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyW

,startDate,onlineExclusive,colorName,unbxd_color_mapping,categoryType,domainUrl,variantUpc,wash,productUrl,parentProductId,keywords,inStoreInventoryCount,category,availability,size,sku,backOrderable,productDescription,catlevel2Name,displaySalePrice,title,description,colorCode,fabrication,productImage,color,categoryAll,valid,limitedQuantity,unbxd_color_for_category,trend,type,newProduct,productId,catlevel3,onlineInventoryCount,is_oos_product,price,rise,catlevel3Name,...,colors,categoryId,parentProduct,legShape,salePrice,giftable,petites,displayPrice,test_colors,listPrice,uniqueId,timeStamp_unbxd,unbxdFeedId,categoryPath1,categoryPath1_fq,categoryPath,3_unbxd_bool,3_unbxd_sku,autosuggest,autosuggest_unstemmed,3_unbxd_text,1_unbxd_text,doctype,2_unbxd_text,_version_,parent_unbxd,category_meta_data,selling_price,promoMessage,promote_metric,onlineExclusivePromoMsg,neckline,collection,styleRefinement,sleeveLength,occasion,fit,pattern,catlevel4,catlevel4Name
0,20190712.0,NO,Pitch Black,Black::https://images.express.com/is/image/exp...,Bottoms,//www.express.com/clothing/women/mid-rise-blac...,0091_07016207_0058,Black,/clothing/women/mid-rise-black-jean-leggings/p...,7016207,"leggings,jean leggings",2916.0,"Extended Sizes,Jeans at $39.90,open,ALL WOMEN'...",YES,"18 Petite,16 Petite,18 Short,16,00 Long,18,10 ...",7016207,NO,The flattering fit of our mid-rise jean leggin...,Jeans,$0.00,mid rise black jean leggings,The flattering fit of our mid-rise jean leggin...,58,Stretch,https://images.express.com/is/image/expressfas...,Black,"Extended Sizes,Jeans at $39.90,open,ALL WOMEN'...",YES,NO,Pitch Black,Clean,Jeans,NO,7016207,"Jeans,Jean Leggings",2972.0,NO,79.9,Mid Rise,Jeans,...,"{""color"": ""Pitch Black"", ""skuUpc"": ""0091_07016...","cat1730036,cat770007,cat1700030,cat4320003,cat...",YES,Legging,$0.00,NO,NO,$79.90,Pitch Black::0091_07016207_0058::True,$79.90,7016207,1584713861308,9dbd1262-0541-45c7-8d56-2efe44e5f3ac,"cat2010027,cat1770018,cat2230008,cat320064,cat...","cat2010027,cat1770018,cat2230008,cat320064,cat...","cat2010027,cat1770018,cat2230008,cat320064,cat...",YES,"91304579,19277745,91304647,91304702,19277721,9...",mid rise black jean leggings,mid rise black jean leggings,"0091_07016207_0058,Jeans","0 Long,16,00 Long,16 Short,18,Legging,Stretch,...",POPULAR_PRODUCTS,"cat1770018,cat320064,cat1910002,cat2005,cat170...",1661692967349387264,True,"{""cat770007"": {""totalProductCount"": 1, ""catego...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20151112.0,YES,"Harvest Yellow,Ivory,burnt amber,Pitch Black,T...",Orange::https://images.express.com/is/image/ex...,Sweaters,//www.express.com/clothing/women/sleeveless-tu...,"0096_09605830_0383,0096_09605830_0058,0096_096...",NaN,/clothing/women/sleeveless-turtleneck-sweater/...,9605830,NaN,213.0,"ALL WOMEN'S REG PRODUCT,Sweaters & Cardigans,D...",YES,"XXS,XL,XXS Petite,M,L Petite,L,XS Petite,S,XL ...",9605830,NO,This form-fitting ribbed sleeveless turtleneck...,Sweaters & Cardigans,$34.93,sleeveless turtleneck sweater,This form-fitting ribbed sleeveless turtleneck...,"1545,3,383,2422,58",NaN,https://images.express.com/is/image/expressfas...,"Purple,Orange,White,Black,Yellow","ALL WOMEN'S REG PRODUCT,Sweaters & Cardigans,D...",YES,NO,"Thornberry,Pitch Black,Ivory,burnt amber,Harve...",NaN,Sweaters,NO,9605830,Sweaters & Cardigans,9902.0,NO,49.9,NaN,Sweaters & Cardigans,...,"{""color"": ""Harvest Yellow"", ""skuUpc"": ""0096_96...","cat4320003,cat2012,cat3970001,cat3960003,cat17...",YES,NaN,$34.93,NO,NO,$49.90,"Harvest Yellow::0096_9605830_2422::False,Pitch...",$49.90,9605830,1584713861308,9dbd1262-0541-45c7-8d56-2efe44e5f3ac,"cat3970001,cat3960003,cat1730035,cat2012,cat43...","cat3970001,cat3960003,cat1730035,cat2012,cat43...","cat3970001,cat3960003,cat1730035,cat2012,cat43...",YES,"19437484,19437781,18309553,82369992,82370020,1...",sleeveless turtleneck sweater,sleeveless turtleneck sweater,"0096_09605830_0383,Sweaters & Cardigans,0096_0...","burnt amber,Thornberry,Black,L,Orange

In [34]:
frames = []
for x in range(0, 1000):
  df_x = process(str(x))
  frames.append(df_x)

final_df = pd.concat(frames, ignore_index = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyW

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:

final_df.to_csv('/content/drive/My Drive/unbxd/unbxd.csv')